In [ ]:
if True:
    %pip install git+https://github.com/arendjan/tmx-pico-aio.git@sensors
    %pip install nest_asyncio
    %pip install pynmea2

Hardware setup:

![button](images/gps_bb.png)

Pico connections:

|Pin # | type  | connection |
|-------|------|------------|
| 18   | GND   |            |
| 7   | GP5  | Pico RX to GPS TX      |
| 36   | 3V3   |  VCC          |



In [ ]:
import asyncio
import sys
import time
from tmx_pico_aio import tmx_pico_aio
import nest_asyncio
import pynmea2
nest_asyncio.apply() # for jupyter notebook asyncio stuff

loop = asyncio.get_event_loop() 

board = tmx_pico_aio.TmxPicoAio(loop=loop)

In [ ]:
current_sentence = ""
items = []
def gps_callback(data): # each few ms, the uart buffer is checked and forwarded. This will be a few bytes, so it needs to be buffered and only when a complete NMEA sentence is received can it be parsed
    global current_sentence
    current_sentence += "".join(map(lambda d: chr(d), data)) # list of ASCII integers to string
    if "\n" in current_sentence: # only parse complete lines
        sentence = current_sentence.splitlines()
        if len(sentence) > 1: # when current_sentence ends with a newline, it will have only one element
            current_sentence = sentence[1]
        else:
            current_sentence = ""
        try: # first sentence is always broken
            msg = pynmea2.parse(sentence[0])
        except:
            return
        # print(msg.fields) # what is in the NMEA message
        # print(msg.name_to_idx) # What/how can we get those fields
        # In jupyter notebooks these prints will be shown after the run_till_end cell
        items.append(msg)
        if msg.sentence_type == "GGA":
            print(msg.latitude, msg.longitude)
        if msg.sentence_type == 'RMC':
            print("Speed", msg.spd_over_grnd)


In [ ]:
async def addSensor(board):
    await asyncio.sleep(1)
    # await board.set_pin_mode_i2c(0, 4, 5)
    await asyncio.sleep(0.1)
    await board.sensors.add_gps(rx=5, tx=4, uart_channel=1, callback=gps_callback) # check the pinout to make sure it is the correct uart channel


loop.run_until_complete(addSensor(board))



In [ ]:
async def run_till_end(board):
    # when to stop?
    await asyncio.sleep(10)
    await board.shutdown() # Otherwise next start will give you errors and you need to restart the kernel
    await asyncio.sleep(1)
    print("done")

loop.run_until_complete(run_till_end(board))

In [ ]:
items